In [4]:
from os import listdir
from os.path import isfile, join
import json
import os
import time
import matplotlib.pyplot as plt 
import pandas as pd
import subprocess

import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import numpy as np
import tqdm
from PIL import Image
import pickle

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from copy import deepcopy
from collections import OrderedDict

In [2]:
%matplotlib inline

In [5]:
project_path = '' #'drive/My Drive/InstaDump'

train_jsons = join(project_path, "train_jsons")
val_jsons = join(project_path, "val_jsons")
photos_for_train = join(project_path, 'photos_for_train')

models_path = join(project_path, 'models_path')

photos_for_train

'photos_for_train'

In [6]:
checked_username = [a[:-5] for a in listdir(train_jsons)]
checked_username[:3], len(checked_username)

(['n0v1_74n', 'vibrationoflight', 'liliannastina'], 763)

In [8]:
name = checked_username[0]

with open(join(train_jsons, f"{name}.json"), 'r') as f:
    username_info = json.loads(f.read()) 

In [9]:
username_info.keys()

dict_keys(['tags', 'post_time', 'likes_number', 'photo_file_path'])

In [16]:
%%time

tags_set = set()
all_tags_list = []

for name in tqdm.tqdm(checked_username):

    with open(f'{train_jsons}/{name}.json', 'r') as f:
        used_usernames_train = json.loads(f.read())
        
    with open(f'{val_jsons}/{name}.json', 'r') as f:
        used_usernames_val = json.loads(f.read())

    for tag_list in used_usernames_train["tags"]:
        tags_set.update(tag_list)
        all_tags_list.extend(tag_list)
        
    for tag_list in used_usernames_val["tags"]:
        tags_set.update(tag_list)
        all_tags_list.extend(tag_list)

100%|██████████| 763/763 [00:00<00:00, 3096.58it/s]

CPU times: user 226 ms, sys: 23.7 ms, total: 250 ms
Wall time: 249 ms


In [25]:
tag_list

[109, 100, 18, 3, 3, 18, 13, 109, 113]

In [17]:
len(all_tags_list), len(tags_set)

(805346, 253)

In [21]:
with open(join(project_path, "tag2idx_245.json"), 'r') as f:
    tag2idx = json.loads(f.read()) 

idx2tag = {}

for k, v in tag2idx.items():
    idx2tag[v] = k

len(tag2idx.keys())

253

In [22]:
tag_freq_dict = Counter()

for tag_idx in tqdm.tqdm(all_tags_list):
    tag_freq_dict[idx2tag[tag_idx]] += 1

tag_freq_dict["love"]

100%|██████████| 805346/805346 [00:00<00:00, 1616987.54it/s]


0

In [23]:
tag_freq_dict

Counter({'ab': 691,
         'accent': 834,
         'actres': 418,
         'all_shot': 3669,
         'amateur': 1641,
         'american': 1233,
         'amstaffs': 20,
         'animal': 2956,
         'animallover': 1163,
         'animegirl': 562,
         'app': 717,
         'architecture': 3583,
         'art': 20314,
         'artist': 3913,
         'asiangirl': 157,
         'babie': 1083,
         'babies': 2288,
         'baby': 4808,
         'bak': 1168,
         'balkan': 558,
         'beam': 63,
         'beautifuldestination': 1679,
         'beautifulplace': 901,
         'bestvacation': 1107,
         'bird': 1101,
         'blackandwhite': 4232,
         'bodybuild': 2696,
         'book': 1176,
         'boot': 1176,
         'bos': 687,
         'box': 330,
         'boy': 2186,
         'build': 763,
         'busines': 1750,
         'cake': 5078,
         'camp': 595,
         'candle': 231,
         'canon': 3650,
         'car': 1431,
         'cat': 4822

In [199]:
smaller_tag_freq_dict = deepcopy(tag_freq_dict)

In [200]:
senseless_words = ['canon', 'foto', 'francemontagne', 'funnyvid', 'funnyvideo', 'goodmorn', 'goodtime',
                  'goodvibe', 'greece', 'greekisland', 'heal', 'indianfoodblogger', 'island', 'landscape_lover',
                   'animallover', 'memorie', 'mama', 'mimysglamanddesign', 'mobile', 'moment', 'montagne',
                  'moodygram', 'movie', 'nikon', 'personalisedgift', 'photographer', 'photographylover', 'mormon',
                  'pic', 'picture', 'platania', 'rowberry', 'salwarsuit', 'seriado', 'size', 'spongebobmeme',
                  'story', 'summervibe', 'thought', 'toptag', 'van', 'vibe', 'video', 'wedge', 'winn', 'print',
                   'wonderful_place', 'wonderfulplace', 'world', 'cool', 'newarrival', 'chailover', 'lago',
                  'happines', 'happy', 'all_shot', 'morn', 'couplegoal', 'goal', 'beautifuldestination',
                  'beautifulplace', 'succes', 'amateur', 'bestvacation', 'nice']

In [202]:
converter = {'ab': 'abs', 'actres': 'actress', 'animegirl': ['anime', 'girl'], "asiangirl": ["asian", 'girl'],
             'babies': 'baby', 'babie': 'baby', 'bak': 'baking', 'cook': ['cook', 'food'],
             'democrat': ['politic', 'democrat'],
             'bodybuild': 'bodybuilding', 'bos':'boss',
             'busines': 'business', 'cats': ['cat', 'pet'], 'cat': ['cat', 'pet'], 
             'cloth': 'clothe', 'colour': 'color', 'dog': ['dog', 'pet'],
             'conservativememe': ['conservative', 'conservativememe', 'meme', 'politic'],
             'cupcake': ['cupcake', 'cake'], 'dankmeme': ['dankmeme', 'meme'], 'dogs': ['dog', 'pet'],
             'draw': ['drawing', 'illustration'], 'drawing': ['drawing', 'illustration'],
             'epicmeme': ['funny', 'meme'], 'fakenew': ['fakenew', 'new'],
             'fashionista': 'fashion', 'fashionstylist' :['fashionstylist', 'style'],
             'fitnes': 'fitness', 'fitnessmodel': ['fitness', 'model'],
             'flowers': 'flower', 'foodpic': 'food', 'freefitnes': ['freefitness', 'fitness'],
             'funnymeme': ["funny", 'meme'], 'gelnail': 'nail', 'hairpiece': 'hair', 
             'hairstyle': ['hair', 'style'], 'homecook': ['home', 'cook', 'food'], 'jean': 'jeans',
             'kids' : 'kid', 'luxuryhome': ['luxury', 'home'], 'nails': 'nail', 'naturelover': 'nature',
             'animallover': 'animal', 'landscape_lover': 'landscape', 'nicekick': 'kick',
             'paint': 'painting', 'pets': 'pet', 'plants': 'plant', 'politicalmeme': ['politic', 'meme'],
             'series': 'serie', 'sports': 'sport', 'styl': 'style', 'trainer': 'train',
             'traveling': 'travel', 'trumpmeme': ["trumpmeme", 'meme', 'politic'], 'tvserie': 'serie', 
             'wedd': ['wedding', 'love'], 'wedding': ['wedding', 'love'], 'relationship': 'love',
             'weightlos': ['weightloss', 'sport'], 'wellnes' : 'sport', 'wildlife': ['wildlife', 'nature'], 
             'yycnail': 'nail', 'catlover': 'cat', 'macaron': ['macaron', 'food'], 'portrait': ['portrait', 'selfie'],
             'eat': 'food', 'childrensbook': 'kid', 'patriot': ['patriot', 'politic'], 'foodie': 'food'
             }

In [203]:
len(all_tags_list)

805346

In [204]:
converted_all_tags_list = []

for idx_tag in tqdm.tqdm(all_tags_list):
    tag = idx2tag[idx_tag]
    
    if tag in senseless_words:
        continue
    
    if tag in converter:
        converted = converter[tag]
        
        if type(converted) == str:
            converted_all_tags_list.append(converted)
        else:
            converted_all_tags_list.extend(converted)
    else:
        converted_all_tags_list.append(tag)
        
        
print(len(converted_all_tags_list))

100%|██████████| 805346/805346 [00:00<00:00, 846881.37it/s]

728165


In [205]:
idx2tag[idx_tag]

'dogs'

In [206]:
converted_all_tags_list[2:5]

['home', 'cook', 'food']

In [207]:
smaller_tag_freq_dict = Counter()

for tag in tqdm.tqdm(converted_all_tags_list):
    smaller_tag_freq_dict[tag] += 1

smaller_tag_freq_dict["nature"]

100%|██████████| 728165/728165 [00:00<00:00, 1726147.56it/s]


43907

In [208]:
len(smaller_tag_freq_dict)

152

In [209]:
'love' in smaller_tag_freq_dict

True

In [210]:
smaller_tag_freq_dict.most_common(300)

[('nature', 43907),
 ('food', 43255),
 ('fashion', 42641),
 ('travel', 40721),
 ('style', 28804),
 ('meme', 23006),
 ('art', 20314),
 ('pet', 20097),
 ('smile', 18717),
 ('politic', 17402),
 ('selfie', 16794),
 ('girl', 16279),
 ('fitness', 13475),
 ('model', 12216),
 ('friend', 11815),
 ('sport', 11412),
 ('landscape', 11172),
 ('dog', 9145),
 ('baby', 8179),
 ('conservative', 8117),
 ('republican', 7769),
 ('holiday', 7396),
 ('cat', 6928),
 ('hair', 6735),
 ('street', 6716),
 ('sunset', 6599),
 ('fit', 6359),
 ('color', 6307),
 ('look', 6266),
 ('cake', 6117),
 ('flower', 5971),
 ('nail', 5871),
 ('christmas', 5609),
 ('illustration', 5290),
 ('love', 5260),
 ('shopping', 5154),
 ('kid', 5060),
 ('new', 4904),
 ('dankmeme', 4806),
 ('portrait', 4476),
 ('outfit', 4472),
 ('wedding', 4452),
 ('plant', 4446),
 ('dessert', 4394),
 ('vintage', 4374),
 ('trend', 4350),
 ('blackandwhite', 4232),
 ('painting', 4220),
 ('home', 3995),
 ('health', 3942),
 ('garden', 3927),
 ('shoe', 3919),
 

In [211]:
n_tags_threshold = 600
tags_above_threshold = {}

for k, v in smaller_tag_freq_dict.items():
    tags_above_threshold[k] = v
    
len(tags_above_threshold)

152

In [212]:
tag2idx_small = {}

for n, tag in enumerate(tags_above_threshold.keys()):
    tag2idx_small[tag] = n
    
len(tag2idx_small), tag2idx_small['interior']

(152, 132)

In [218]:
with open(join(project_path, 'tag2idx_small.json'), 'w') as f:
    json.dump(tag2idx_small, f)

In [213]:
small_train_jsons = join(project_path, 'small_train_jsons')
small_val_jsons = join(project_path, 'small_val_jsons')

In [221]:
lost_data = 0
saved_data = 0

val_share = 0.1
min_number_of_items = 10

for name in tqdm.tqdm(checked_username):

    with open(f'{train_jsons}/{name}.json', 'r') as f:
        used_usernames_train = json.loads(f.read())
    
    user_converted_info = {'tags': [], 'photo_file_path': [], 'likes_number': []}
    good_idx = []
    good_tags_lists = []
    
    photo_file_path_list = []
    likes_number_list = []
        
    for n, tag_idx_list in enumerate(used_usernames_train["tags"]):
        tag_list = [idx2tag[idx] for idx in tag_idx_list]
        good_tags = []
    
        for tag in tag_list:
            if tag in senseless_words:
                continue
    
            if tag in converter:
                converted = converter[tag]

                if type(converted) == str:
                    good_tags.append(converted)
                else:
                    good_tags.extend(converted)
            else:
                good_tags.append(tag)
            
        unique_tags = list(set(good_tags))
        n_tags = len(unique_tags)
            
        if n_tags > 0:
            good_tags_lists.append(unique_tags)
            good_idx.append(n)
        else:
            lost_data += 1
        
    for idx in good_idx:
        user_converted_info['photo_file_path'].append(used_usernames_train["photo_file_path"][idx])
        user_converted_info['likes_number'].append(used_usernames_train["likes_number"][idx])    
    
    tags_idx_converted_list = [[tag2idx_small[tag] for tag in tag_list] for tag_list in good_tags_lists]
    
    user_converted_info['tags'].extend(tags_idx_converted_list)
    
    
    with open(f'{val_jsons}/{name}.json', 'r') as f:
        used_usernames_val = json.loads(f.read())
        
    good_idx = []
    good_tags_lists = []
        
    for n, tag_idx_list in enumerate(used_usernames_train["tags"]):
        tag_list = [idx2tag[idx] for idx in tag_idx_list]
        good_tags = []
    
        for tag in tag_list:
            if tag in senseless_words:
                continue
    
            if tag in converter:
                converted = converter[tag]

                if type(converted) == str:
                    good_tags.append(converted)
                else:
                    good_tags.extend(converted)
            else:
                good_tags.append(tag)
            
        unique_tags = list(set(good_tags))
        n_tags = len(unique_tags)
            
        if n_tags > 0:
            good_tags_lists.append(unique_tags)
            good_idx.append(n)
            lost_data += 1
        
    for idx in good_idx:
        user_converted_info['photo_file_path'].append(used_usernames_train["photo_file_path"][idx])
        user_converted_info['likes_number'].append(used_usernames_train["likes_number"][idx])   
        
    tags_idx_converted_list = [[tag2idx_small[tag] for tag in tag_list] for tag_list in good_tags_lists]
    user_converted_info['tags'].extend(tags_idx_converted_list)
    
    
    items_number = len(user_converted_info['likes_number'])
    
    if items_number < min_number_of_items:
        lost_data += 1
        continue
    
    val_unfo = {'tags': [], 'photo_file_path': [], 'likes_number': []}
    train_unfo = {'tags': [], 'photo_file_path': [], 'likes_number': []}
    
    val_samples_number = int(items_number * val_share)
    val_idx = np.random.choice(items_number, val_samples_number, replace=False)
    
    saved_data += items_number
    
    for idx in range(items_number):
        if idx in val_idx:
            val_unfo['tags'].append(user_converted_info['tags'][idx])
            val_unfo['photo_file_path'].append(user_converted_info['photo_file_path'][idx])
            val_unfo['likes_number'].append(user_converted_info['likes_number'][idx])
            
        else:
            train_unfo['tags'].append(user_converted_info['tags'][idx])
            train_unfo['photo_file_path'].append(user_converted_info['photo_file_path'][idx])
            train_unfo['likes_number'].append(user_converted_info['likes_number'][idx])
    
    with open(join(small_train_jsons, f'{name}.json'), 'w') as f:
        json.dump(train_unfo, f)
    
    with open(join(small_val_jsons, f'{name}.json'), 'w') as f:
        json.dump(val_unfo, f)

100%|██████████| 763/763 [00:04<00:00, 152.66it/s]


In [222]:
lost_data, saved_data

(133045, 264126)

In [197]:
len(val_unfo['likes_number']), len(train_unfo['likes_number']), len(user_converted_info['likes_number'])

(9, 83, 92)